In [118]:
import itertools
import operator
import re
from abc import *
from copy import deepcopy
from operator import itemgetter
from typing import *
import numpy as np
import scipy as sp
import networkx as nx
import mne
import time
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.learning.informativeness import CrossInformativeness, Informativeness, SubjectsInformativeness, \
    NodesInformativeness
from nodestimation.learning.networking import sparse_graph, graph_to_hemispheres, hemispheres_division_modularity, \
    metric_for_hemispheres
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nibabel
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon, spearmanr, pearsonr
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft
from sklearn.linear_model import LinearRegression

ENGEL1 = [
    'B1C2',
    'B1R1',
    'G1R1',
    'G1V2',
    'J1T2',
    'K1V1',
    'L1P1',
    'M1G2',
    'M1N2',
    'O1O2',
    'R1D2',
    'S1A2',
    'S1B1',
    'S1H1',
    'S1U3'
]
ENGEL2 = [
    'L2M1',
    'M2S2',
    'N2K2',
    'P1H2'
]
ENGEL3 = [
    'N3S2',
    'S3R1'
]
ENGEL4 = [
    'K4L2'
]
REJECTED = [
    'S1U3',
    'P1H2'
]

AGE = {
    'B1C2': 28.0,
    'B1R1': 16.0,
    'G1R1': 23.0,
    'G1V2': 5.0,
    'J1T2': 8.0,
    'K1V1': 10.0,
    'K4L2': 14.0,
    'L1P1': 16.0,
    'L2M1': 20.0,
    'M1G2': 8.0,
    'M1N2': 7.0,
    'M2S2': 18.0,
    'N2K2': 30.0,
    'N3S2': 10.0,
    'O1O2': 18.0,
    'R1D2': 6.5,
    'P1H2': 7.0,
    'S1A2': 12.0,
    'S1B1': 17.0,
    'S1H1': 28.0,
    'S3R1': 19.0,
    'S1U3': 15.0,
}

SEX = {
    'B1C2': 'f',
    'B1R1': 'm',
    'G1R1': 'f',
    'G1V2': 'm',
    'J1T2': 'f',
    'K1V1': 'f',
    'K4L2': 'f',
    'L1P1': 'f',
    'L2M1': 'f',
    'M1G2': 'm',
    'M1N2': 'm',
    'M2S2': 'm',
    'N2K2': 'm',
    'N3S2': 'm',
    'O1O2': 'f',
    'R1D2': 'f',
    'P1H2': 'm',
    'S1A2': 'm',
    'S1B1': 'm',
    'S1H1': 'm',
    'S3R1': 'm',
    'S1U3': 'f',
}

ENGEL = dict()
for group, i in zip([ENGEL1, ENGEL2, ENGEL3, ENGEL4], range(4)):
    ENGEL.update({
        subject_name: i+1
        for subject_name in group
    })


def depth(coords: np.ndarray, center: np.ndarray = np.array([0, 0, 0])) -> float:
    return np.sqrt(
        (coords[0] - center[0]) ** 2 +
        (coords[1] - center[1]) ** 2 +
        (coords[2] - center[2]) ** 2
    )

In [2]:
subjects = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=(4, 8),
    centrality_metrics=['eigen', 'between', 'degree', 'info']
    )

All computation has been already done, loading of the existing file with the solution...


In [3]:
DATASET = metric_for_hemispheres(subjects, nx.algorithms.global_efficiency)
print('graph done')

P1H2: DONE, RUNTIME: 2.684141159057617
M2S2: DONE, RUNTIME: 2.7546422481536865
R1D2: DONE, RUNTIME: 3.2327709197998047
N3S2: DONE, RUNTIME: 2.0280492305755615
S1A2: DONE, RUNTIME: 2.2443552017211914
S1H1: DONE, RUNTIME: 2.6411516666412354
K1V1: DONE, RUNTIME: 2.6209235191345215
L1P1: DONE, RUNTIME: 1.9068398475646973
M1G2: DONE, RUNTIME: 2.4962594509124756
G1V2: DONE, RUNTIME: 1.7862520217895508
G1R1: DONE, RUNTIME: 1.814648151397705
M1N2: DONE, RUNTIME: 2.057861804962158
S1B1: DONE, RUNTIME: 2.251647710800171
S1U3: DONE, RUNTIME: 1.8322336673736572
B1R1: DONE, RUNTIME: 1.8558673858642578
S3R1: DONE, RUNTIME: 2.1729493141174316
N2K2: DONE, RUNTIME: 3.03715443611145
K4L2: DONE, RUNTIME: 2.2112114429473877
B1C2: DONE, RUNTIME: 2.2130000591278076
J1T2: DONE, RUNTIME: 2.242374897003174
O1O2: DONE, RUNTIME: 2.171424388885498
L2M1: DONE, RUNTIME: 2.3755929470062256
graph done


In [4]:
dataset = DATASET.copy()

for sample in dataset.index:
    subject = sample[:4]
    if subject in REJECTED:
        dataset = dataset.drop(index=sample)

In [5]:
cross_hemispheres_informativeness = CrossInformativeness()
cross_subjects_informativeness = CrossInformativeness()

for _ in range(100):
    hemispheres_informatoveness = Informativeness()
    subjects_informativeness = SubjectsInformativeness()
    acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()

    for i in range(100):

        y = dataset['resected'].to_numpy()
        x = dataset[[f'global_efficiency_for_wpli_4-8Hz', f'global_efficiency_for_envelope_4-8Hz']].to_numpy()

        scaler = StandardScaler()
        x = scaler.fit_transform(x)

        samples = [[sample] for sample in dataset.index.tolist()]

        x = np.append(x, samples, axis=1)
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        train_samples, test_samples = x_train[:, 2], x_test[:, 2]
        x_train, x_test = x_train[:, 0:2], x_test[:, 0:2]

        clf = svm.SVC()
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)

        for predicted, actual, sample, value in zip(pred, y_test, test_samples, x_test):
            hemispheres_informatoveness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'
            subjects_informativeness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'

    cross_subjects_informativeness.informativeness = subjects_informativeness
    cross_hemispheres_informativeness.informativeness = hemispheres_informatoveness

In [8]:
deepest_regions = dict()
n_regions = dict()

for subject in subjects:
    if subject.name in REJECTED:
        continue
    else:
        deepest_regions.update({
            subject.name: np.min(
                np.array([
                    depth(node.center_coordinates)
                    for node in subject.nodes
                    if node.type == 'resected'
                ])
            )
        })
        size = 0

        for node in subject.nodes:
            if node.type == 'resected':
                size += 1

        n_regions.update({
            subject.name: size
        })

max = np.max(np.array(list(deepest_regions.values())))
min = np.min(np.array(list(deepest_regions.values())))

deepest_regions_norm = {
    item[0]: 1 - (item[1] - min)/(max - min)
    for item in deepest_regions.items()
}

In [141]:
series = list()
names = [
        'accuracy',
        'depth',
        'size',
        'age',
        'sex',
        'engel',
    ]

age = AGE.copy()
engel = ENGEL.copy()
sex = SEX.copy()

for subject_name in REJECTED:
    age.pop(subject_name)
    engel.pop(subject_name)
    sex.pop(subject_name)

for data in [
        cross_subjects_informativeness.acc(),
        deepest_regions_norm,
        n_regions,
        age,
        sex,
        engel
    ]:
    series.append(
        dict(
            sorted(
                data.items(),
                key=lambda item: item[0]
            )
        )
    )

df = pd.DataFrame(
    series,
    index=names
)
df = df.T
df['sex'] = [int(sample == 'f') for sample in df['sex']]
print(df)
# df.to_csv('~/Documents/Samples_informativeness&Rejected(1).csv')

sep = '\n\n*****************\n'
print(sep)
for param in ['engel', 'age', 'size', 'depth']:

    s_corr, s_p = spearmanr(df[param].to_numpy(), df['accuracy'].to_numpy())
    p_corr, p_p = pearsonr(df[param].to_numpy(), df['accuracy'].to_numpy())
    print(f'Correlation between {param} and accuracy\n'
          f'SpearmannCorrCoef: {s_corr}\nSpearmann p-val: {s_p}\n'
          f'PearsonCorrCoef: {p_corr}\nPearson p-val: {p_p}{sep}')

      accuracy     depth size   age  sex engel
B1C2   0.45768       0.0    5  28.0    1     1
B1R1  0.980802  0.527525    2  16.0    0     1
G1R1       1.0  0.557838    2  23.0    1     1
G1V2  0.501629  0.562176    2   5.0    0     1
J1T2  0.732012  0.560677    1   8.0    1     1
K1V1  0.491632  0.626464    6  10.0    1     1
K4L2  0.376052   0.48751    4  14.0    1     4
L1P1  0.980373  0.890547    4  16.0    1     1
L2M1  0.434024  0.845401    1  20.0    1     2
M1G2  0.775568  0.315033    4   8.0    0     1
M1N2  0.978488  0.580463    2   7.0    0     1
M2S2  0.903023   0.59526    1  18.0    0     2
N2K2   0.92126  0.475333    1  30.0    0     2
N3S2  0.019708  0.780576    4  10.0    0     3
O1O2       1.0  0.098071    1  18.0    1     1
R1D2  0.977393  0.612838    1   6.5    1     1
S1A2  0.461038  0.489974    1  12.0    0     1
S1B1  0.698227  0.620951    4  17.0    0     1
S1H1  0.461115       1.0    2  28.0    0     1
S3R1    0.5102  0.998972    6  19.0    0     3


***********

In [142]:
for param in ['age', 'size', 'depth']:

    s_corr, s_p = spearmanr(df[param].to_numpy(), df['engel'].to_numpy())
    p_corr, p_p = pearsonr(df[param].to_numpy(), df['engel'].to_numpy())
    print(f'Correlation between {param} and engel\n'
          f'SpearmannCorrCoef: {s_corr}\nSpearmann p-val: {s_p}\n'
          f'PearsonCorrCoef: {p_corr}\nPearson p-val: {p_p}{sep}')

Correlation between age and engel
SpearmannCorrCoef: 0.2603240227748078
Spearmann p-val: 0.26764952825435895
PearsonCorrCoef: 0.08850571603123218
Pearson p-val: 0.7105979409293223

*****************

Correlation between size and engel
SpearmannCorrCoef: 0.07531013707577049
Spearmann p-val: 0.7523369714575918
PearsonCorrCoef: 0.2707589773689651
Pearson p-val: 0.24824114204835782

*****************

Correlation between depth and engel
SpearmannCorrCoef: 0.18220933561228697
Spearmann p-val: 0.44197242329595254
PearsonCorrCoef: 0.2613828454102226
Pearson p-val: 0.26563797230976827

*****************



In [105]:
X = np.array([
    [d, a]
    for d, a in zip(
        df['depth'].to_numpy(),
        df['age'].to_numpy()
    )
])
y = df[['accuracy']].to_numpy()

reg = LinearRegression().fit(X, y)


print('Linear model for depth and age to accuracy')
print(f'score: {reg.score(X, y)}')
print(f'coef: {reg.coef_}')
print(f'intercept: {reg.intercept_}')

Linear model for depth and age to accuracy
score: 0.05329913268194075
coef: [[0.00647801 0.00146337]]
intercept: [0.30349674]


In [146]:
X = np.array([
    [d, s, a, e, s]
    for d, s, a, e in zip(
        df['depth'].to_numpy(),
        df['size'].to_numpy(),
        df['age'].to_numpy(),
        df['engel'].to_numpy(),
    )
])
y = df[['accuracy']].to_numpy()

reg = LinearRegression().fit(X, y)


print('Linear model for depth, size, age, engel to accuracy')
print(f'score: {reg.score(X, y)}')
print(f'coef: {reg.coef_}')
print(f'intercept: {reg.intercept_}')

Linear model for depth, size, age, engel to accuracy
score: 0.34948441640988204
coef: [[-0.09920302 -0.02400556  0.00283855 -0.12800614 -0.02400556]]
intercept: [1.01782093]


In [148]:
X = np.array([
    [s, a]
    for s, a in zip(
        df['size'].to_numpy(),
        df['engel'].to_numpy()
    )
])
y = df[['accuracy']].to_numpy()

reg = LinearRegression().fit(X, y)


print('Linear model for size and engel to accuracy')
print(f'score: {reg.score(X, y)}')
print(f'coef: {reg.coef_}')
print(f'intercept: {reg.intercept_}')

Linear model for size and engel to accuracy
score: 0.3356995658663029
coef: [[-0.04893646 -0.13282272]]
intercept: [1.01437373]


In [153]:
X = np.array([
    [s, d, a]
    for s, d, a in zip(
        df['size'].to_numpy(),
        df['depth'].to_numpy(),
        df['age'].to_numpy(),
    )
])
y = df[['engel']].to_numpy()

reg = LinearRegression().fit(X, y)


print('Linear model for size and depth to engel')
print(f'score: {reg.score(X, y)}')
print(f'coef: {reg.coef_}')
print(f'intercept: {reg.intercept_}')


Linear model for size and depth to engel
score: 0.1376372787961052
coef: [[0.12534618 0.8303841  0.01135654]]
intercept: [0.50086548]
